In [20]:
import json
import os
import os.path as osp
import sys
from PIL import Image
import pickle as pkl
import cv2

from detectron.datasets.json_dataset import JsonDataset
from detectron.utils.io import load_object, save_object
from detectron.datasets.roidb import combined_roidb_for_training
import detectron.utils.vis as vis_utils
from detectron.core.config import cfg
import time
date = time.strftime("%Y%m%d")



def checkMkdir(dirname):
    if not osp.isdir(dirname):
        os.makedirs(dirname)
        
dataset = 'FashionV2_train' #'OpenImage_train'
boxFilePath = '/media/data/chnxi/GOD/threshold_bboxes/'
mapFilePath = '/media/data/chnxi/GOD/mappings/src_nameid_to_GOD_id_mappings/'
outAnnoFolder = '/media/data/chnxi/GOD/json_annotations/'
outAnnoFile = osp.join(outAnnoFolder, 'tmp_{}_annotations.json'.format(dataset))

visPath = '/media/data/chnxi/GOD/visualizations/'
visFolder = osp.join(visPath, dataset, date)
checkMkdir(visFolder)

modelSetMap = {'Fashion_Detector'  : 'fashion',
               'HF_Detector'       : 'furniture',
               'COCO_Mask_Detector': 'COCO',
               'OpenImage_Detector': 'Open Image'}

if dataset == 'furniture_train':
    gtSetName = 'furniture'
    mapFile = mapFilePath + 'HF_dataset_sources_ids.pkl'
    srcFiles = {'Fashion_Detector'  : 'furniture_train.json',
                'COCO_Mask_Detector': 'furniture_train.pkl',
                'OpenImage_Detector': 'furniture_train.pkl'}
elif dataset == 'FashionV2_train':
    gtSetName = 'fashion'
    mapFile = mapFilePath + 'Fashion_dataset_sources_ids.pkl'
    srcFiles = {'HF_Detector'       : 'FashionV2_train.pkl',
                'COCO_Mask_Detector': 'FashionV2_train.pkl',
                'OpenImage_Detector': 'FashionV2_train.pkl'}
elif dataset == 'OpenImage_train':
    gtSetName = 'Open Image'
    mapFile = mapFilePath + 'OpenImage_dataset_sources_ids.pkl'
    srcFiles = {'Fashion_Detector'  : 'OpenImage_train.json',
                'HF_Detector'       : 'OpenImage_train.pkl',
                'COCO_Mask_Detector': 'OpenImage_train.pkl'}
elif dataset == 'COCO_trainval':
    gtSetName = 'COCO'
    mapFile = mapFilePath + 'COCO_dataset_sources_ids.pkl'
    srcFiles = {'Fashion_Detector'  : 'COCO_trainval.json',
                'HF_Detector'       : 'coco_2014_trainval.pkl',
                'OpenImage_Detector': 'COCO_trainval.pkl'}

# for modelName in srcFiles:
#     box_file_name = srcFiles[modelName]
#     boxFileName = osp.join(boxFilePath, modelName, box_file_name)
#     print ("boxFileName = {}".format(boxFileName))
    
    
    

In [14]:
if dataset in ['furniture_train', 'FashionV2_train', 'OpenImage_train']:
    print ("creating json dataset {}".format(dataset))
    ds = JsonDataset(dataset)
    roidb = ds.get_roidb(gt=True)
elif dataset == 'COCO_trainval':
    datasetNames = ('coco_2014_train','coco_2014_valminusminival')
    roidb = []
    for dsName in datasetNames:
        ds = JsonDataset(dsName)
        rdb = ds.get_roidb(gt=True)
        roidb = roidb + rdb
imgNum = len(roidb)
print ('imgNum = {}'.format(imgNum))

creating json dataset FashionV2_train
================= loading annotation FashionV2_train
loading annotations into memory...
Done (t=3.54s)
creating index...
index created!
self.subset_categories = None
getting roidb
Done (t=134.22s)
imgNum = 160916


In [15]:
print (roidb[0]['image'])
print (roidb[-1]['image'])

/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/FashionV2/Images/b17ed5bd46400f89300cc903349e4c3e.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/FashionV2/Images/81e22eebfc07d3fc81f886fa0d8544a5.jpg


In [16]:
print ("loading {}".format(mapFile))
src_god_map = pkl.load(open(mapFile,'rb'))
print (src_god_map.keys())
cocoMap = src_god_map['name_to_god_id']['COCO']
print (cocoMap)
cocoIdMap = src_god_map['id_map']['COCO']
print(cocoIdMap)
print (len(cocoMap))

loading /media/data/chnxi/GOD/mappings/src_nameid_to_GOD_id_mappings/Fashion_dataset_sources_ids.pkl
dict_keys(['id_map', 'name_to_god_id'])
{'teddy bear': 541, 'bicycle': 546, 'apple': 281, 'laptop': 112, 'tennis racket': 514, 'donut': 263, 'snowboard': 509, 'carrot': 287, 'zebra': 82, 'keyboard': 109, 'scissors': 478, 'fire hydrant': 245, 'mouse': 111, 'airplane': 542, 'boat': 564, 'kite': 499, 'sheep': 77, 'horse': 58, 'cup': 338, 'traffic light': 570, 'toaster': 140, 'bowl': 334, 'book': 464, 'elephant': 51, 'orange': 296, 'stop sign': 568, 'knife': 342, 'pizza': 275, 'fork': 340, 'hair drier': 129, 'sandwich': 278, 'umbrella': 237, 'banana': 284, 'bus': 547, 'toothbrush': 407, 'spoon': 352, 'giraffe': 53, 'cell phone': 148, 'broccoli': 285, 'frisbee': 492, 'remote': 153, 'surfboard': 512, 'cow': 47, 'hot dog': 268, 'car': 548, 'skateboard': 507, 'dog': 49, 'cat': 45, 'baseball glove': 488, 'skis': 508, 'train': 559, 'truck': 553, 'parking meter': 453, 'suitcase': 161, 'cake': 251,

In [17]:
print ("Loading model bboxes!")
srcBoxes = {}
for modelName in srcFiles:
    box_file_name = srcFiles[modelName]
    boxFileName = osp.join(boxFilePath, modelName, box_file_name)
    print ("Loading boxFileName = {}".format(boxFileName))
    if modelName in ['COCO_Mask_Detector', 'HF_Detector']:
        # format: all_boxes[imgid][class_id]
        all_boxes = load_object(boxFileName)
        print ("loaded")
        print ("all_boxes.size = ({}, {})".format(len(all_boxes), len(all_boxes[0])))
        srcBoxes[modelName] = all_boxes
    elif modelName == 'OpenImage_Detector':
        # all_boxes[file_name] = {'file_name': file_name,
        #        'boxes': boxes,
        #        'label_names': label_names,
        #        'labels': labels}
        all_boxes = load_object(boxFileName)
        print ("len(all_boxes) = {}".format(len(all_boxes)))
        srcBoxes[modelName] = all_boxes
    elif '.json' in box_file_name:
        # {'image_id': 'd7a0b91d2d83d457', 
        #'boxes': [{'score': 0.228, 'category_id': 2, 
        #'bbox': [506.49920000000003, 414.4128, 657.8248, 544.1536], 
        #'category_name': 'accessories-glasses'}], 'image_file_name': 'd7a0b91d2d83d457.jpg'}
        print ("loading {}".format(boxFileName))
        dets = json.load(open(boxFileName,'r'))
        all_boxes = {}
        for ix, det in enumerate(dets):
            # make it a dictionary with image file name as key
            img_file_name = det['image_file_name']
            all_boxes[img_file_name] = det['boxes']
        print ("len(all_boxes) = {}".format(len(all_boxes)))
        print ("loaded")
        srcBoxes[modelName] = all_boxes #dets
    print ("srcBoxes[{}] loaded".format(modelName))

Loading model bboxes!
Loading boxFileName = /media/data/chnxi/GOD/threshold_bboxes/HF_Detector/FashionV2_train.pkl
loaded
all_boxes.size = (160916, 58)
srcBoxes[HF_Detector] loaded
Loading boxFileName = /media/data/chnxi/GOD/threshold_bboxes/COCO_Mask_Detector/FashionV2_train.pkl
loaded
all_boxes.size = (160916, 80)
srcBoxes[COCO_Mask_Detector] loaded
Loading boxFileName = /media/data/chnxi/GOD/threshold_bboxes/OpenImage_Detector/FashionV2_train.pkl
len(all_boxes) = 160916
srcBoxes[OpenImage_Detector] loaded


In [18]:
imgName = '2988531ACD30C78DC4C1F1EA859ED1C02DDF7773F9CBC67FFAD71F25A2CAD0D688A35E26D72C7525A0C4100ED0C9C3CA008942A59F6B9173D1E3552387B19AF3.jpg'
print ("srcBoxes['COCO_Mask_Detector'][10][14][1] = {}\n".format(srcBoxes['COCO_Mask_Detector'][10][14][1]))
print ("srcBoxes['Fashion_Detector'][imgName] = {}\n".format(srcBoxes['Fashion_Detector'][imgName]))
print ("srcBoxes['OpenImage_Detector'][imgName] = {}\n".format(srcBoxes['OpenImage_Detector'][imgName]))

IndexError: list index out of range

In [19]:
#############   Creating taxonomy   #############################################
import pandas as pd
labelFile = '/media/data/chnxi/GOD/taxonomy/GOD_taxonomy_V1.tsv'
god_data = pd.read_csv(labelFile, delimiter='\t')
god_name_id_map = dict(zip(god_data['GOD_v1_name'], god_data['GOD_v1_id']))
god_classes = list(god_data['GOD_v1_name'])
print ("len(god_classes) original = {}".format(len(god_classes)))
god_classes = ['__background__'] + god_classes

len(god_classes) original = 579


In [21]:
data = {}
supercat = 'Generic'
data['info'] = {'description':'Bing Generic Object Detection dataset on {}'.format(dataset)}
data['type'] = 'instances'

data['categories'] = []

for cls, id in god_name_id_map.items():
    cats = {'supercategory':supercat,
            'id':id,
            'name':cls}
    data['categories'].append(cats)

print (data['categories'][:10])

[{'supercategory': 'Generic', 'id': 1, 'name': 'animal\\amphibian\\frog'}, {'supercategory': 'Generic', 'id': 2, 'name': 'animal\\arthropod\\arachnid\\scorpion'}, {'supercategory': 'Generic', 'id': 3, 'name': 'animal\\arthropod\\arachnid\\spider'}, {'supercategory': 'Generic', 'id': 4, 'name': 'animal\\arthropod\\crustacean\\crab'}, {'supercategory': 'Generic', 'id': 5, 'name': 'animal\\arthropod\\crustacean\\lobster'}, {'supercategory': 'Generic', 'id': 6, 'name': 'animal\\arthropod\\crustacean\\shrimp'}, {'supercategory': 'Generic', 'id': 7, 'name': 'animal\\arthropod\\insect\\ant'}, {'supercategory': 'Generic', 'id': 8, 'name': 'animal\\arthropod\\insect\\bee'}, {'supercategory': 'Generic', 'id': 9, 'name': 'animal\\arthropod\\insect\\butterfly'}, {'supercategory': 'Generic', 'id': 10, 'name': 'animal\\arthropod\\insect\\caterpillar'}]


In [22]:
imgNum = len(roidb)
# imgIds = range(imgNum)

In [24]:
import matplotlib.pyplot as plt
color_list = ['g', 'r', 'b', 'm']
def visImageClsThrshs(plt, splot, im, imgBoxes, ecolor='m', title='', showCateg=False):
    #plt.cla()
    fx, fy, fi = splot
    plt.subplot(fx, fy, fi)
    plt.imshow(im)
    plt.axis('off')
    plt.title(title)
    ax = plt.gca()
    for obj in imgBoxes:
        #print obj
        bbox = obj['bbox']
        categ_id = obj['category_id']
        categ = god_classes[categ_id].split('\\')[-1]
        #ecolor = 'm' #if gid == jmax else gtcolor
        #print ("{}: bbox = {}".format(categ, bbox))
        ax.add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2],
                          bbox[3], fill=False,
                          edgecolor=ecolor, linestyle='-', linewidth=1))
        if showCateg:
            ax.text(bbox[0], bbox[1]-2, '{}'.format(categ), bbox={'facecolor': ecolor, 'alpha': 0.5})
    plt.draw()


def getAnno(image_id, bbox, categ_id, box_id):
    boxw = float(bbox[2]-bbox[0]) # convert np.ndarray.float32 to float in case json.dump not serializable
    boxh = float(bbox[3]-bbox[1])
    nbbox = [float(bbox[0]), float(bbox[1]), boxw, boxh]
    anno = {'segmentation':[],
            'area': boxw*boxh,
            'iscrowd':0,
            'image_id':image_id,
            'bbox':nbbox, # because COCO format [l, t, w,h] #float(r),float(b)],
            'category_id':categ_id,
            'id':box_id
            }
#     print ("anno.segmentation.type = {}".format(type(anno['segmentation'])))
#     print ("anno.bbox.type = {}".format(type(anno['bbox'])))
#     print ("anno.area.type = {}".format(type(anno['area'])))
#     print ("anno.iscrowd.type = {}".format(type(anno['iscrowd'])))
#     print ("anno.image_id.type = {}".format(type(anno['image_id'])))
#     print ("anno.category_id.type = {}".format(type(anno['category_id'])))
#     print ("anno.id.type = {}".format(type(anno['id'])))
    return anno

# from random import randint
# dsIdx = randint(0,imgNum-1)
# print(dsIdx)
# imgIds = [dsIdx]
# print (imgIds)

import random
visNum = 100
imgIds = random.sample(range(len(roidb)), visNum)
print(imgIds)

vis = True
saveVis = True
totalBoxCnt = 0
data["images"] = []
data["annotations"] = []

for imgid in imgIds: #imgid, entry in enumerate(roidb):
    visAnno = {} 
    entry = roidb[imgid]
    ###############################################
    ## data['images']
    ###############################################
    file_name = entry['image']
    imgName = osp.basename(file_name)
    imgH = entry['height']
    imgW = entry['width']
    img = {'file_name': imgName,
           'height': float(entry['height']),
           'width': float(entry['width']),
           'id':imgid
           }
    data["images"].append(img)
    print (img)
    print ("img.file_name.type = {}".format(type(img['file_name'])))
    print ("img.height.type = {}".format(type(img['height'])))
    print ("img.width.type = {}".format(type(img['width'])))
    print ("img.id.type = {}".format(type(img['id'])))
    ###############################################
    ## Add GT Boxes
    ###############################################
    src_god_id_map = src_god_map['id_map'][gtSetName]
    src_name_godid_map = src_god_map['name_to_god_id'][gtSetName]
    visName = gtSetName + '_GT'
    visAnno[visName] = []
    gt_boxes = entry['boxes']
    gt_src_cls_ids = entry['gt_classes']
    for bid, box in enumerate(gt_boxes):
        src_class_id = gt_src_cls_ids[bid]
        if src_class_id in src_god_id_map:
            god_class_id = src_god_id_map[src_class_id]
            print ("======{}:{} ==> GOD:{} {}".format(gtSetName, src_class_id, god_class_id, god_classes[god_class_id]))
            anno = getAnno(imgid, box[:4], god_class_id, totalBoxCnt)
            #print (anno)
            data["annotations"].append(anno)
            if vis:
                visAnno[visName].append(anno)
            totalBoxCnt += 1
    ###############################################
    ## Add boxes from other models
    ###############################################
    for modelName in srcFiles:
        modelSetName = modelSetMap[modelName]
        src_god_id_map = src_god_map['id_map'][modelSetName]
        src_name_godid_map = src_god_map['name_to_god_id'][modelSetName]
        visAnno[modelName] = []
#         print ("{} ==> {}".format(modelName, modelSetName))
        if modelName in ['COCO_Mask_Detector', 'HF_Detector']:
            # format: all_boxes[imgid][class_id]
            for cid, cls_boxes in enumerate(srcBoxes[modelName][imgid]):
                src_class_id = cid + 1 # all_boxes starts from 0
                if src_class_id in src_god_id_map and len(cls_boxes) > 0:
                    god_class_id = src_god_id_map[src_class_id]
                    print ("========{}:{} ==> GOD:{}:{}".format(modelSetName, src_class_id, god_class_id, god_classes[god_class_id]))
                    for bbox in cls_boxes:
                        anno = getAnno(imgid, bbox, god_class_id, totalBoxCnt)
                        #if totalBoxCnt % 10 == 0:
                        #print (anno)
                        data["annotations"].append(anno)
                        if vis:
                            visAnno[modelName].append(anno)
                        totalBoxCnt += 1
        if modelName == 'OpenImage_Detector':
            # all_boxes[file_name] = {'file_name': file_name,
            #        'boxes': boxes,
            #        'label_names': label_names,
            #        'labels': labels}
            boxes = srcBoxes[modelName][imgName]['boxes']
            label_names = srcBoxes[modelName][imgName]['label_names']
            for bid, box in enumerate(boxes):
                src_class_name = label_names[bid] # do not use labels: it's not correct for OpenImage Id
                if src_class_name in src_name_godid_map:
                    god_class_id = src_name_godid_map[src_class_name]
                    print ("========= {}:{} ==> GOD:{}:{}".format(modelSetName, src_class_name, god_class_id, god_classes[god_class_id]))
                    bbox = [box[0] * imgW, box[1] * imgH, box[2] * imgW, box[3] * imgH]
                    anno = getAnno(imgid, bbox, god_class_id, totalBoxCnt)
                    #print (anno)
                    data["annotations"].append(anno)
                    if vis:
                        visAnno[modelName].append(anno) 
                    totalBoxCnt += 1                 
        if modelName == 'Fashion_Detector': #'.json' in box_file_name:
            # all_boxes[imgName] = [ {[506.49920000000003, 414.4128, 657.8248, 544.1536], 
            #'category_name': 'accessories-glasses'} ]
            if imgName in srcBoxes[modelName]:
                boxes = srcBoxes[modelName][imgName]
                for bid, obj in enumerate(boxes):
                    src_class_name = obj['category_name']
                    if src_class_name in src_name_godid_map:
                        print (obj)
                        god_class_id = src_name_godid_map[src_class_name]
                        print ("========{}:{} ==> GOD:{}{}".format(modelSetName, src_class_name, god_class_id, god_classes[god_class_id]))
                        bbox = obj['bbox']
                        anno = getAnno(imgid, bbox, god_class_id, totalBoxCnt)
                        #print (anno)
                        data["annotations"].append(anno)
                        if vis:
                            visAnno[modelName].append(anno) 
                        totalBoxCnt += 1    

    #################################
    # visualization of merging boxes
    #################################
    if vis:
        im = Image.open(file_name)
        f, axarr = plt.subplots(1, 4, sharey=True, figsize=(20, 20))
        plt.cla()
        fid = 0
        for modelName in visAnno: #['prec_at_0.9']: #new_cls_thrsh_dict:
            print (modelName)
            ax = axarr[fid]
            visImageClsThrshs(plt, (1,4, fid+1), im, visAnno[modelName], ecolor=color_list[fid], title=modelName, showCateg=True)
            fid += 1
        if saveVis: # and len(visAnno['COCO_Mask_Detector']) > 0:
            plt.savefig(osp.join(visFolder,imgName), bbox_inches='tight',pad_inches = 0)
        plt.show()
        plt.close()

[16561, 143319, 129023, 85205, 89121, 49370, 42566, 76491, 38653, 58919, 16155, 63864, 89466, 10208, 150603, 137360, 111959, 95567, 133101, 44677, 73689, 41770, 146377, 76925, 84518, 51513, 67419, 109570, 87589, 69034, 24448, 72385, 128711, 155654, 140289, 89617, 1379, 132516, 21796, 114113, 138269, 73337, 13506, 28705, 103772, 150186, 57327, 30817, 31469, 72511, 120797, 129069, 4625, 72126, 95072, 24120, 20523, 83343, 124110, 113977, 14332, 157126, 63629, 15170, 159401, 112459, 30467, 30949, 155500, 99024, 24451, 12889, 142935, 113889, 59049, 53266, 13245, 62886, 38172, 22943, 29300, 71411, 61444, 82659, 59414, 100743, 140492, 137282, 146164, 41099, 48369, 131970, 128194, 37389, 125809, 126020, 136039, 156767, 139330, 76157]
{'file_name': '444eb577978732dd8a20cb0b2801b3f5.jpg', 'height': 477.0, 'width': 352.0, 'id': 16561}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:35 ==> GOD:198 fashi

COCO_Mask_Detector
OpenImage_Detector
{'file_name': '00aa27312945192f8984bc814130b30c.jpg', 'height': 300.0, 'width': 300.0, 'id': 111959}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:33 ==> GOD:196 fashion\jewelry\bracelet
fashion_GT
HF_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '60b23c2cda25581d4515fd1fbf6441f9.jpg', 'height': 200.0, 'width': 200.0, 'id': 95567}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:57 ==> GOD:219 fashion\shoes\wedges
fashion_GT
HF_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'a01cce382c42486aa145248aec58a8c3.jpg', 'height': 677.0, 'width': 316.0, 'id': 133101}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:33 ==> GOD:196 fashion\jewelry

COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'aa8b8a48fcc6f25aea47be021a057542.jpg', 'height': 2400.0, 'width': 700.0, 'id': 1379}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:28 ==> GOD:178 fashion\dresses\formal_dress
fashion_GT
HF_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '6473045b3cba0febf26c7f6a5cede582.jpg', 'height': 300.0, 'width': 300.0, 'id': 132516}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:2 ==> GOD:183 fashion\glasses
======fashion:33 ==> GOD:196 fashion\jewelry\bracelet
======fashion:35 ==> GOD:198 fashion\jewelry\necklace
======fashion:4 ==> GOD:188 fashion\head_wear\hat
======fashion:22 ==> GOD:171 fashion\bottom_wear\shorts
========COCO:68 ==> GOD:148:electronics\phone\cell_phone
========= Open Image:sunglasses ==> GOD:185:fashion\glasses\sunglas

HF_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'b395015686e0375c44c7b11b5be77b5a.jpg', 'height': 500.0, 'width': 375.0, 'id': 120797}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:33 ==> GOD:196 fashion\jewelry\bracelet
fashion_GT
HF_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'd3506ccf17154e5647f7368db4bf39f3.jpg', 'height': 934.0, 'width': 640.0, 'id': 129069}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:33 ==> GOD:196 fashion\jewelry\bracelet
======fashion:14 ==> GOD:160 fashion\bag\purse
======fashion:51 ==> GOD:214 fashion\shoes\heels
======fashion:51 ==> GOD:214 fashion\shoes\heels
======fashion:2 ==> GOD:183 fashion\glasses
======fashion:34 ==> GOD:197 fashion\jewelry\earrings
======fashion:1 ==> GOD:164 fashion\belt
======fashion:33 ==> GOD:196 fashio

HF_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'a2955f2c6c9eaf9bbdcb9923f5ea5704.jpg', 'height': 308.0, 'width': 240.0, 'id': 30949}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:45 ==> GOD:208 fashion\outer_wear\sweatshirt
======fashion:23 ==> GOD:172 fashion\bottom_wear\skirt
========= Open Image:miniskirt ==> GOD:173:fashion\bottom_wear\skirt\miniskirt
========= Open Image:miniskirt ==> GOD:173:fashion\bottom_wear\skirt\miniskirt
fashion_GT
HF_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'b7174792be29a935e1b78b32efa38a17.jpg', 'height': 853.0, 'width': 640.0, 'id': 155500}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:13 ==> GOD:159 fashion\bag\messenger
======fashion:23 ==> GOD:172 fashion\bottom_wear\skirt
======fashion:50 ==> GOD:213 fashion\shoes\flats
=

HF_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'a39600120fee381a7beaff841a1d08e9.jpg', 'height': 480.0, 'width': 480.0, 'id': 61444}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:34 ==> GOD:197 fashion\jewelry\earrings
======fashion:34 ==> GOD:197 fashion\jewelry\earrings
fashion_GT
HF_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'd2ce3288febc442eee1ddab991270f9a.jpg', 'height': 960.0, 'width': 639.0, 'id': 82659}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:23 ==> GOD:172 fashion\bottom_wear\skirt
fashion_GT
HF_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '2c017eff0047a042ed9abfa4f673e811.jpg', 'height': 1514.0, 'width': 721.0, 'id': 59414}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
im

HF_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '9d149c03f0d251e9674eeb60ec0e7a83.jpg', 'height': 600.0, 'width': 600.0, 'id': 76157}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======fashion:14 ==> GOD:160 fashion\bag\purse
fashion_GT
HF_Detector
COCO_Mask_Detector
OpenImage_Detector


In [11]:
# print (data['images'][:10])
# print ("Saving annotatinos to {}".format(outAnnoFile))
# with open(outAnnoFile,'w') as fout:
#     json.dump(data, fout)
# print ("Done")

[{'file_name': 'E3CCB304F9566D553A169B7279415E647902A57D5DEB4C1CA2E6939423381DCA0EC64DAB84F27F2D9A842E9C0C58BA58B7BBE36E34BE649E460A844E71D6B1E3.jpg', 'height': 733.0, 'width': 550.0, 'id': 31665}, {'file_name': '76FC7A5A958D86F7696689F40FAAE48AC7E5631307E63F51B25E26841FCCF86F6C48244AEBB6230096EF71323FC3B50BE1308BB9895756B24E424CF8C5B9BF44.jpg', 'height': 490.0, 'width': 736.0, 'id': 55449}, {'file_name': '78DB239735C6AD1931281C4F366F7692E8B096AEBDF2836FB292B41ECD644E0BD14540BCD628C16EF9A6D42AB998CFA80EADEBA0C3E1E485467F1FAFF90D2ECC.jpg', 'height': 768.0, 'width': 511.0, 'id': 55372}, {'file_name': 'E4E7B811946426F7D813F74DFB26EB883F48EF64A3D244C9261112B2C1BB79380663E92B894CE3BCE9DF9ECA43528F8248CB9A44F626FA3557B62CE8EF0FAD34.jpg', 'height': 363.0, 'width': 242.0, 'id': 79517}, {'file_name': '968B5C47AB681195EB993915BBA536FE00893D2028AB33C3C3880B8D0B83BFD442980835396B0C4A1A4BE4034D65054AD4B9AB4B86952082D98ADE2D1C9CA99B.jpg', 'height': 400.0, 'width': 327.0, 'id': 102645}, {'file_name':